In [6]:
import pandas as pd
import os
import torch 
import clip
import numpy as np
from PIL import Image
import string
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device,"device")

cuda device


In [7]:
summary_bert_path = "data/Summaries/Summary_Bert_77.csv"
summary_bert = pd.read_csv(summary_bert_path)


summary_bert_ingredients_path="data/Summaries/export_summary_bert_with_ingredients.csv"
summary_bert_ingredients=pd.read_csv(summary_bert_ingredients_path)

data_dir = "data"
images_dir = "Food Images/Food Images"

In [12]:
recipe = summary_bert["Summary_Bert_77"]
print("number of sumary recipe ",len(recipe)) #2067
recipe = recipe[:200]
print("we only take",len(recipe))

recipe_tensor = clip.tokenize(recipe, truncate=True).to(device)
print(recipe_tensor.shape)

tensor_bert = torch.tensor(recipe_tensor).to(device)
torch.save(tensor_bert, "data/tensors/bert_tensor.pt")


liste_images = summary_bert["Image_Name"].tolist()
liste_images=liste_images[:200]
liste_images = [image + ".jpg" for image in liste_images]


images = os.listdir(os.path.join(data_dir, images_dir)) #13582 images in total 
print("number of images in total ",len(images))

number of sumary recipe  12710
we only take 200
torch.Size([200, 77])
number of images in total  13582


/tmp/ipykernel_1081346/2356681205.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_bert = torch.tensor(recipe_tensor).to(device)


In [11]:
models = clip.available_models()
print(models)
model, preprocess = clip.load('RN50', device,jit=False)

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [5]:
images_inputs=[]

for im in liste_images:
        im_path = os.path.join(data_dir, images_dir, im)
        #images_inputs.append(preprocess(Image.open(im_path)))
        images_inputs.append(im_path)
images_inputs_tensor = torch.tensor((images_inputs)).to(device)
torch.save(images_inputs_tensor, "data/Tensors_data/images_inputs_tensor.pt")
print(len(images_inputs))

TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

# With bert Summary

In [9]:
print("text array shape",recipe.shape)
print("text_tensor shape", recipe_tensor.shape)
#print("images_inputs_tensor shape ", images_inputs_tensor.shape)


print("images array shape",len(images_inputs))

#we ensure that we have the same order for the images and the text
print(recipe[:2])
print(images_inputs[:2])


text array shape (200,)
text_tensor shape torch.Size([200, 77])
images array shape 200
0    Roast chicken in a large castiron skillet and ...
1    Roast the potatoes for 15 to 20 minutes depend...
Name: Summary_Bert_77, dtype: object
['data/Food Images/Food Images/miso-butter-roast-chicken-acorn-squash-panzanella.jpg', 'data/Food Images/Food Images/crispy-salt-and-pepper-potatoes-dan-kluger.jpg']


In [39]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


image = preprocess(Image.open("data/Food Images/Food Images/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813.jpg")).unsqueeze(0).to(device)


#print("image",image.shape) #torch.Size([1, 3, 224, 224])
#print("text",recipe_tensor.shape) #torch.Size([200, 77])

with torch.no_grad():
   
    image_features = model.encode_image(image)
    #print("type image_features",type(image_features.shape)) #<class 'torch.Size'>
    #print("shape image_features",image_features.shape) #torch.Size([1, 1024]
    
    text_features = model.encode_text(recipe_tensor)
    #print("type text_features",type(text_features.shape)) #<class 'torch.Size'>
    #print("shape text_features",text_features.shape) #torch.Size([200, 1024]

    
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
#print(similarity[0])

values, indices = torch.topk(similarity[0], k=5)

# Convert the tensors to numpy arrays
values = values.cpu().numpy()
indices = indices.cpu().numpy()

# Create a list of tuples containing the values and indices
values_indices = list(zip(values, indices))

# Sort the list in descending order based on the values
values_indices.sort(reverse=True)

# Extract the sorted values and indices
sorted_values, sorted_indices = zip(*values_indices)

max_index=sorted_indices[0]


# Print the sorted values and indices
for value, index in zip(sorted_values, sorted_indices):
    print("Value:", value , "for index :" ,index)



print(max_index)
print(recipe[max_index])




Value: 0.4458 for index : 126
Value: 0.1257 for index : 189
Value: 0.0979 for index : 78
Value: 0.0751 for index : 91
Value: 0.04346 for index : 96
126
Toast pistachios in a dry small skillet over mediumlow heat tossing occasionally until


In [64]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

good_answer=0
bad_answer=0

print(type(images_inputs))

for i,img in enumerate(images_inputs):
    if i>10:
        break

    image = preprocess(Image.open(img)).unsqueeze(0).to(device)

    with torch.no_grad():
    
        image_features = model.encode_image(image)
        text_features = model.encode_text(recipe_tensor)
    
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)


    values, indices = torch.topk(similarity[0], k=5)

    # Convert the tensors to numpy arrays
    values = values.cpu().numpy()
    indices = indices.cpu().numpy()

    # Create a list of tuples containing the values and indices
    values_indices = list(zip(values, indices))

    # Sort the list in descending order based on the values
    values_indices.sort(reverse=True)

    # Extract the sorted values and indices
    sorted_values, sorted_indices = zip(*values_indices)

    max_index=sorted_indices[0] #text qui correspond à l'image selon le modèle 
    
    label=recipe[max_index] #rectte

"""
    for value, index in zip(sorted_values, sorted_indices):
        print("Value:", value , "for index :" ,index)
"""

    print("LABEL is : ",label)
    print("Model Answer",recipe[max_index])

    if label==sorted_values[0]: 
        good_answer+=1
    else: 
        bad_answer+=1


print("good_answer",good_answer)
print("bad_answer",bad_answer)

<class 'list'>
Value: 0.3792 for index : 0
Value: 0.2229 for index : 18
Value: 0.07587 for index : 13
Value: 0.03017 for index : 149
Value: 0.01888 for index : 82
label is Roast chicken in a large castiron skillet and roast on middle rack until an instant
model answer Roast chicken in a large castiron skillet and roast on middle rack until an instant
Value: 0.4275 for index : 101
Value: 0.2324 for index : 1
Value: 0.05106 for index : 106
Value: 0.04794 for index : 65
Value: 0.04233 for index : 129
label is To make the meatballs place the breadcrumbs in a small bowl and cover with
model answer To make the meatballs place the breadcrumbs in a small bowl and cover with
Value: 0.148 for index : 22
Value: 0.09705 for index : 116
Value: 0.0721 for index : 170
Value: 0.0721 for index : 167
Value: 0.0721 for index : 117
label is Preheat the oven to 350°F Butter a 9inch nonstick metal cake pan
model answer Preheat the oven to 350°F Butter a 9inch nonstick metal cake pan
Value: 0.4045 for index 

In [65]:
print("recipe", recipe[101]) 
print("image",images_inputs[101])



recipe To make the meatballs place the breadcrumbs in a small bowl and cover with
image ComputerVision_Data/Food Images/Food Images/chicken-meatballs-with-molokhieh-falastin.jpg


# With ONLY the main ingredients (GPT3.5)

## With the first five rows

In [14]:
# Creation of a tensor for the text 

liste_ingredients=[]

context_length=77
#print(device,"device")
liste_ingredients_tensor = torch.zeros(len(summary_bert_ingredients), context_length, dtype=torch.long)
labels = []

torch.save(liste_ingredients_tensor, "data/tensors/ingredients_tensor.pt")

for i, row in enumerate(summary_bert_ingredients.iterrows()):
    
    summary = row[1]["summary_with_ingredients"]
    ingredients = summary.split()[-2:]
    
    ingredients=str(ingredients[0])+ " " +str(ingredients[1])

    to_supress = string.punctuation + "—" + "–" + "()"
    ingredients = ingredients.translate(str.maketrans("", "",to_supress)) # Remove punctuation from summary

    liste_ingredients.append(ingredients)  # Access row data using integer indices
    labels.append(row[1]["Image_Name"])  # Access row data using integer indices

   
print(liste_ingredients[:5])
print(labels[:5])



['minutesWhole chicken', 'whites potatoes', 'cream cheese', 'turkeyBread cubes', 'sugar bourbon']
['miso-butter-roast-chicken-acorn-squash-panzanella', 'crispy-salt-and-pepper-potatoes-dan-kluger', 'thanksgiving-mac-and-cheese-erick-williams', 'italian-sausage-and-bread-stuffing-240559', 'newtons-law-apple-bourbon-cocktail']


In [84]:
#TEST with the 5 first line

main_ingr= ['minutesWhole chicken', 'whites potatoes', 'cream cheese', 'turkeyBread cubes', 'sugar bourbon']
label = ['miso-butter-roast-chicken-acorn-squash-panzanella', 'crispy-salt-and-pepper-potatoes-dan-kluger', 'thanksgiving-mac-and-cheese-erick-williams', 'italian-sausage-and-bread-stuffing-240559', 'newtons-law-apple-bourbon-cocktail']

In [85]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

image = preprocess(Image.open("/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/italian-sausage-and-bread-stuffing-240559.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(main_ingr).to(device)
print(type(text))


with torch.no_grad():
   
    image_features = model.encode_image(image)

    
    text_features = model.encode_text(text)


# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
print(similarity[0])

values, indices = torch.topk(similarity[0], k=5)

# Convert the tensors to numpy arrays
values = values.cpu().numpy()
indices = indices.cpu().numpy()

# Create a list of tuples containing the values and indices
values_indices = list(zip(values, indices))

# Sort the list in descending order based on the values
values_indices.sort(reverse=True)

# Extract the sorted values and indices
sorted_values, sorted_indices = zip(*values_indices)

max_index=sorted_indices[0]


# Print the sorted values and indices
for value, index in zip(sorted_values, sorted_indices):
    print("Value:", value , "for index :" ,index)


print(max_index)
print(liste_ingredients[max_index])
print(array_text[max_index])





<class 'torch.Tensor'>
tensor([0.1512, 0.2739, 0.0020, 0.5620, 0.0108], device='cuda:0',
       dtype=torch.float16)
Value: 0.562 for index : 3
Value: 0.274 for index : 1
Value: 0.1512 for index : 0
Value: 0.01079 for index : 4
Value: 0.001995 for index : 2
3
turkeyBread cubes
Bread can be toasted 3 days ahead and kept once cool in a sealed bag at room 


## With all the rows 

In [83]:


image_to_print=[]

image = preprocess(Image.open("/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/caramelized-plantain-parfait.jpg")).unsqueeze(0).to(device)

text = clip.tokenize(liste_ingredients).to(device)
print(type(text))

with torch.no_grad():
   
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)



# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
print(similarity[0])

values, indices = torch.topk(similarity[0], k=5)

# Convert the tensors to numpy arrays
values = values.cpu().numpy()
indices = indices.cpu().numpy()

# Create a list of tuples containing the values and indices
values_indices = list(zip(values, indices))

# Sort the list in descending order based on the values
values_indices.sort(reverse=True)

# Extract the sorted values and indices
sorted_values, sorted_indices = zip(*values_indices)

max_index=sorted_indices[0]


# Print the sorted values and indices
for value, index in zip(sorted_values, sorted_indices):
    print("Value:", value , "for index :" ,index)


print(max_index)
print(liste_ingredients[max_index])
print(array_text[max_index])


<class 'torch.Tensor'>
tensor([1.4353e-04, 2.4414e-04, 3.1531e-05,  ..., 6.0737e-05, 2.0325e-05,
        1.0118e-03], device='cuda:0', dtype=torch.float16)
Value: 0.10645 for index : 58
Value: 0.0755 for index : 919
Value: 0.03403 for index : 1004
Value: 0.03099 for index : 906
Value: 0.0305 for index : 116
58
yolks kefir
Melt butter over medium heat in a large skillet Add pine nuts and cook until 


"\nTo verify the image \n\nfor i in range(len(labels)): \n    if i==index: \n        image_to_print.append(labels[i])\n\n\n# Display the images using matplotlib\nfig, axes = plt.subplots(1, len(image_to_print), figsize=(15, 3), squeeze=False)\n\nfor i, image_name in enumerate(image_to_print):\n    image_path = os.path.join(images_dir, image_name)\n    image = Image.open(image_path)\n    axes[0, i].imshow(image)\n    axes[0, i].set_title(image_name)\n    axes[0, i].axis('off')\n\nplt.show()\n"

# With all ingredients 

## For the first 5 rows

In [15]:
ingr_text=[]
labels = []
context_length=77

liste_ingredients_tensor = torch.zeros(len(summary_bert), context_length, dtype=torch.long)


for i, row in enumerate(summary_bert.iterrows()):
    ingredients = row[1]["Cleaned_Ingredients"]

    to_supress = string.punctuation + "—" + "–" + "()"
    ingredients = ingredients.translate(str.maketrans("", "",to_supress)) # Remove punctuation from summary
    ingredients = ''.join([char for char in ingredients if not char.isdigit()])
    ingredients = ingredients.replace("Tbsp", "").replace("½", "").replace("¾", "").replace("lb", "").replace("tsp", "").replace("⅓", "").replace("  ", "").replace("¼", "")

    #bug with ingredients 
    if len(ingredients)>context_length:
        ingredients = ingredients[:context_length]

    ingr_text.append(ingredients)  # Access row data using integer indices
    labels.append(row[1]["Image_Name"])  # Access row data using integer indices

    liste_ingredients_tensor[i] = clip.tokenize([ingredients]).to(device)
    

torch.save(liste_ingredients_tensor, "data/tensors/All_ingredients_tensor.pt")
 

ingr=ingr_text[:5]
label=labels[:5]

print(ingr)
print(label)


['whole chicken kosher salt divided plus moresmall acorn squash about total fin', ' large egg whitespound new potatoes aboutinch in diameterteaspoons kosher sal', ' cup evaporated milkcup whole milk garlic powder onion powder smoked paprika ', 'to pound round Italian loaf cut into inch cubescupstablespoons olive oil divi', ' teaspoon dark brown sugarteaspoon hot water oz bourbonoz fresh lemon juicete']
['miso-butter-roast-chicken-acorn-squash-panzanella', 'crispy-salt-and-pepper-potatoes-dan-kluger', 'thanksgiving-mac-and-cheese-erick-williams', 'italian-sausage-and-bread-stuffing-240559', 'newtons-law-apple-bourbon-cocktail']


In [103]:
image_to_print=[]

image = preprocess(Image.open("/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/italian-sausage-and-bread-stuffing-240559.jpg")).unsqueeze(0).to(device)

text = clip.tokenize(ingr).to(device)
print(type(text))


with torch.no_grad():
   
    image_features = model.encode_image(image)
   
    text_features = model.encode_text(text)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
print(similarity[0])

values, indices = torch.topk(similarity[0], k=5)

# Convert the tensors to numpy arrays
values = values.cpu().numpy()
indices = indices.cpu().numpy()

# Create a list of tuples containing the values and indices
values_indices = list(zip(values, indices))

# Sort the list in descending order based on the values
values_indices.sort(reverse=True)

# Extract the sorted values and indices
sorted_values, sorted_indices = zip(*values_indices)

max_index=sorted_indices[0]


# Print the sorted values and indices
for value, index in zip(sorted_values, sorted_indices):
    print("Value:", value , "for index :" ,index)


print(max_index)
print(ingr_text[max_index])
print(array_text[max_index])


<class 'torch.Tensor'>
tensor([0.0359, 0.0140, 0.0468, 0.8823, 0.0208], device='cuda:0',
       dtype=torch.float16)
Value: 0.8823 for index : 3
Value: 0.04678 for index : 2
Value: 0.03586 for index : 0
Value: 0.02075 for index : 4
Value: 0.014046 for index : 1
3
to pound round Italian loaf cut into inch cubescupstablespoons olive oil divi
Bread can be toasted 3 days ahead and kept once cool in a sealed bag at room 


## with all the rows 

In [14]:
import os
image_to_print=[]
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

image = preprocess(Image.open("/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/italian-sausage-and-bread-stuffing-240559.jpg")).unsqueeze(0).to(device)
liste_ingredients_tensor = torch.load("/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Tensors_data/All_ingredients_tensor.pt").to(device)
print(liste_ingredients_tensor.shape) #Huge tensor 

liste_ingredients_tensor=liste_ingredients_tensor[:1000]

with torch.no_grad():
   
    image_features = model.encode_image(image)
   
    text_features = model.encode_text(liste_ingredients_tensor)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)


values, indices = torch.topk(similarity[0], k=5)

# Convert the tensors to numpy arrays
values = values.cpu().numpy()
indices = indices.cpu().numpy()

# Create a list of tuples containing the values and indices
values_indices = list(zip(values, indices))

# Sort the list in descending order based on the values
values_indices.sort(reverse=True)

# Extract the sorted values and indices
sorted_values, sorted_indices = zip(*values_indices)

max_index=sorted_indices[0]


# Print the sorted values and indices
for value, index in zip(sorted_values, sorted_indices):
    print("Value:", value , "for index :" ,index, "for recipe : ",labels[index])


print(max_index)
print(ingr_text[max_index])
print(array_text[max_index])



torch.Size([13463, 77])


OutOfMemoryError: CUDA out of memory. Tried to allocate 152.00 MiB. GPU 0 has a total capacity of 11.74 GiB of which 117.94 MiB is free. Including non-PyTorch memory, this process has 11.51 GiB memory in use. Of the allocated memory 11.17 GiB is allocated by PyTorch, and 214.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)